In [1]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

env = make_vec_env(
    "HalfCheetah-v4",
    rng=np.random.default_rng(),
    n_envs=4,
    env_make_kwargs={"render_mode": "rgb_array"},
)


expert = load_policy(
    "sac-huggingface",
    organization="sb3",
    env_name="HalfCheetah-v3",
    venv=env,
)

C:\Users\xavier\anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: 'bytes' object cannot be interpreted as an integer
  warnings.warn(
C:\Users\xavier\anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:95: UserWarning: You loaded a model that was trained using OpenAI Gym. We strongly recommend transitioning to Gymnasium by saving that model again.
  warnings.warn(


In [2]:
from src.models.dagger import TrajectoryAccumulator
from src.models.dagger import generate_trajectories

In [12]:
obs = env.reset()
print(obs.shape)
print(env.num_envs,env.observation_space.shape[0] ,env.action_space.shape[0])

(4, 17)
4 17 6


In [4]:
traj = generate_trajectories(expert, env)

partial_trajectories {0: [], 1: [], 2: [], 3: []}
Start generate Traj
finish_trajectory
finish_trajectory
finish_trajectory
finish_trajectory
active env sum  0
active env [False False False False]
END


In [13]:
print(len(traj[0]))
print(len(traj[1]))
print(len(traj[2]))
print(len(traj[3]))

1001
1001
1001
1001
